# Replicating Fama & French factors

The Fama and French three-factor model [Fama1993](https://doi.org/2329112) is a cornerstone of asset pricing. On top of the market factor represented by the traditional CAPM beta, the model includes the size and value factors. We introduce both factors in the previous chapter, and their definition remains the same. Size is the SMB factor (small-minus-big) that is long small firms and short large firms. The value factor is HML (high-minus-low) and is long in high book-to-market firms and short the low book-to-market counterparts. In this chapter, we also want to show the main idea of how to replicate these significant factors.

We use CRSP and Compustat as data sources, as we need exactly the same variables to compute the size and value factors in the way Fama and French do it. Hence, there is nothing new below and we only load data from our `SQLite`-database introduced in our chapter on *"Accessing & managing financial data"*.

In [1]:
import pandas as pd
import sqlite3
from datetime import timedelta
# Read sqlite query results into a pandas DataFrame
tidy_finance = sqlite3.connect("D:/Tidy/tidyfinance.sqlite")
crsp_monthly = pd.read_sql_query("SELECT * from crsp_monthly", tidy_finance)
factors_ff_monthly = pd.read_sql_query("SELECT * from factors_ff_monthly", tidy_finance)
compustat = pd.read_sql_query("SELECT * from compustat", tidy_finance)

In [2]:
data_ff=pd.merge(crsp_monthly,factors_ff_monthly,left_on='month',right_on='month',how='left')[['permno', 'gvkey', 'month', 'ret_excess', 'mkt_excess','mktcap', 'mktcap_lag', 'exchange']]
data_ff=data_ff.assign(month=pd.to_datetime(data_ff['month']),permno = data_ff['permno'].astype(int))
data_ff=data_ff.dropna()
be = compustat[[ 'gvkey', 'datadate', 'be']].dropna()
be['month']=pd.to_datetime(be['datadate']) + timedelta(days=1) - pd.DateOffset(months=1)

## Data preparation

Yet when we start merging our data set for computing the premiums, there are a few differences to the previous chapter. First, Fama and French form their portfolios in June of year $t$, whereby the returns of July are the first monthly return for the respective portfolio. For firm size, they consequently use the market capitalization recorded for June. It is then held constant until June of year $t+1$.

Second, Fama and French also have a different protocol for computing the book-to-market ratio. They use market equity as of the end of year $t - 1$ and the book equity reported in year $t-1$, i.e., the `datadate` is within the last year. Hence, the book-to-market ratio can be based on accounting information that is up to 18 months old. Market equity also does not necessarily reflect the same time point as book equity.

To implement all these time lags, we again employ the temporary `sorting_date`-column. Notice that when we combine the information, we want to have a single observation per year and stock since we are only interested in computing the breakpoints held constant for the entire year.

In [3]:
me_ff=data_ff.loc[data_ff.month.dt.month == 6].assign(sorting_date = data_ff.month +pd.DateOffset(months=1),me_ff = data_ff.mktcap)[['permno', 'sorting_date', 'me_ff']]

In [4]:
me_ff_dec=data_ff.loc[data_ff.month.dt.month == 12].assign(sorting_date =  data_ff.loc[data_ff.month.dt.month == 12].month.dt.year.apply(lambda x: pd.to_datetime(str(x+1)+'0701')),bm_me = data_ff.mktcap) [['permno', 'gvkey', 'sorting_date', 'bm_me']]

In [5]:
bm_ff=be.assign(sorting_date = be.month.dt.year.apply(lambda x: pd.to_datetime(str(x+1)+'0701')), bm_be = be.be)[['gvkey', 'sorting_date', 'bm_be']].dropna().merge(me_ff_dec, left_on = ["gvkey", "sorting_date"],right_on = ["gvkey", "sorting_date"],how='inner')

In [6]:
bm_ff=bm_ff.assign(bm_ff = bm_ff.bm_be / bm_ff.bm_me)[['permno', 'sorting_date', 'bm_ff']]

In [7]:
variables_ff = me_ff.merge(bm_ff, left_on=["permno", "sorting_date"],right_on = ["permno", "sorting_date"],how='inner').dropna()

## Portfolio sorts

Next, we construct our portfolios with an adjusted `assign_portfolio()` function. Fama and French rely on NYSE-specific breakpoints, they form two portfolios in the size dimension at the median and three portfolios in the dimension of book-to-market at the 30%- and 70%-percentiles, and they use independent sorts.

In [8]:
portfolios_ff=variables_ff.merge(data_ff, left_on = ["permno" , "sorting_date"],right_on = ["permno" , "month"],how='inner')

In [9]:
nyse_sz=portfolios_ff.loc[portfolios_ff['exchange'].isin(["NYSE"])].groupby(['month'])['me_ff'].median().to_frame().reset_index().rename(columns={'me_ff':'sizemedn'})

In [10]:
nyse_bm=portfolios_ff.loc[portfolios_ff['exchange'].isin(["NYSE"])].groupby(["month"])['bm_ff'].describe(percentiles=[0.3, 0.7]).reset_index()

In [11]:
nyse_bm=nyse_bm[['month','30%','70%']].rename(columns={'30%':'bm30', '70%':'bm70'})

In [12]:
nyse_breaks = pd.merge(nyse_sz, nyse_bm, how='inner', on=['month'])

In [13]:
ccm1_jun = pd.merge(portfolios_ff, nyse_breaks, how='left', on=['month'])

In [14]:
import numpy as np
def sz_bucket(row):
    if row['me_ff'] == np.nan:
        value=''
    elif row['me_ff']<=row['sizemedn']:
        value=1
    else:
        value=2
    return value

def bm_bucket(row):
    if 0<=row['bm_ff']<=row['bm30']:
        value = 1
    elif row['bm_ff']<=row['bm70']:
        value=2
    elif row['bm_ff']>row['bm70']:
        value=3
    else:
        value=''
    return value

In [15]:
ccm1_jun['portfolio_me']=np.where((ccm1_jun['bm_ff']>0)&(ccm1_jun['me_ff']>0), ccm1_jun.apply(sz_bucket, axis=1), '')
ccm1_jun['portfolio_bm']=np.where((ccm1_jun['bm_ff']>0)&(ccm1_jun['me_ff']>0), ccm1_jun.apply(bm_bucket, axis=1), '')

In [16]:
portfolios_ff=ccm1_jun[['permno', 'sorting_date', 'portfolio_me', 'portfolio_bm']]

Next, we merge the portfolios to the return data for the rest of the year. To implement this step, we create a new column `sorting_date` in our return data by setting the date to sort on to July of $t-1$ if the month is June (of year $t$) or earlier or to July of year $t$ if the month is July or later.

In [17]:
data_ff.loc[data_ff.month.dt.month <= 6,'sorting_date'] = data_ff.month.dt.year.apply(lambda x: pd.to_datetime(str(x-1)+'0701'))

In [18]:
data_ff.loc[data_ff.month.dt.month >= 7,'sorting_date'] = data_ff.month.dt.year.apply(lambda x: pd.to_datetime(str(x)+'0701'))

In [19]:
portfolios_ff=data_ff.merge(portfolios_ff, left_on=["permno",'sorting_date'],right_on=["permno",'sorting_date'],how='inner')

## Fama and French factor returns

Equipped with the return data and the assigned portfolios, we can now compute the value-weighted average return for each of the six portfolios. Then, we form the Fama and French factors. For the size factor (i.e., SMB), we go long in the three small portfolios and short the three large portfolios by taking an average across either group. For the value factor (i.e., HML), we go long in the two high book-to-market portfolios and short the two low book-to-market portfolios, again weighting them equally.

In [20]:
portfolios_ff['portfolio']=portfolios_ff.apply(lambda x:str(int(x['portfolio_me'])) + '-' + str(int(x['portfolio_bm'])),axis = 1)

In [21]:
portfolios_ff=portfolios_ff.groupby(['month','portfolio']).apply(lambda x: pd.Series([np.average(x['ret_excess'], weights=x['mktcap_lag'])], 
                                                                index=['ret'])).reset_index()

In [22]:
portfolios_ff['portfolio_me']=portfolios_ff.portfolio.apply(lambda x:x[0])
portfolios_ff['portfolio_bm']=portfolios_ff.portfolio.apply(lambda x:x[2])

In [23]:
smb_replicated=portfolios_ff.groupby(['month','portfolio_me']).ret.mean().unstack()
smb_replicated=(smb_replicated['1']-smb_replicated['2']).rename('smb_r')

In [24]:
hml_replicated=portfolios_ff.groupby(['month','portfolio_bm']).ret.mean().unstack()
hml_replicated=(hml_replicated['3']-hml_replicated['1']).rename('hml_r')

## Replication evaluation

In the previous section, we replicated the size and value premiums following the procedure outlined by Fama and French. However, we did not follow their procedure strictly. The final question is then: how close did we get? We answer this question by looking at the two time-series estimates in a regression analysis using `statsmodels.api`. If we did a good job, then we should see a non-significant intercept (rejecting the notion of systematic error), a coefficient close to 1 (indicating a high correlation), and an adjusted R-squared close to 1 (indicating a high proportion of explained variance).

In [25]:
factors_ff_monthly['month']=pd.to_datetime(factors_ff_monthly['month'])

In [26]:
test = factors_ff_monthly.merge( hml_replicated, left_on = ["month"],right_on=["month"]).merge( smb_replicated, left_on = ["month"],right_on=["month"]) 

In [27]:
test['hml_r']=test['hml_r'].round(decimals = 4)
test['smb_r']=test['smb_r'].round(decimals = 4)

In [28]:
import statsmodels.api as sm
y=test['smb'].values
X=test['smb_r'].values
X=sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                 4.498e+04
Date:                Tue, 23 Aug 2022   Prob (F-statistic):               0.00
Time:                        16:16:09   Log-Likelihood:                 2971.4
No. Observations:                 714   AIC:                            -5939.
Df Residuals:                     712   BIC:                            -5930.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002      0.000     -1.250      0.2

In [29]:
import statsmodels.api as sm
y=test['hml'].values
X=test['hml_r'].values
X=sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                 1.359e+04
Date:                Tue, 23 Aug 2022   Prob (F-statistic):               0.00
Time:                        16:16:13   Log-Likelihood:                 2603.1
No. Observations:                 714   AIC:                            -5202.
Df Residuals:                     712   BIC:                            -5193.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.000      1.739      0.0

The evidence hence allows us to conclude that we did a relatively good job in replicating the original Fama-French premiums, although we cannot see their underlying code. 
From our perspective, a perfect match is only possible with additional information from the maintainers of the original data.